In [1]:
# load data from the sgdb.db which is a sqlite db into a data frame
import pandas as pd
import sqlite3
import os
import sys

# adds parent directory to sys.path
from pathlib import Path
import json
sys.path.append(str(Path(os.getcwd()).resolve().parent))  # Add the parent directory to the path

from src.preprocess.names import NameProcessor


In [2]:

data_dir = '/Users/syamil/Projects/searchgov/data'
names_parquet = os.path.join(data_dir, 'names.parquet')
FORCE_RELOAD = True
if not os.path.exists(names_parquet) or FORCE_RELOAD:
    # Load the names from the sqlite database and save them to a parquet file
    conn = sqlite3.connect(os.path.join(data_dir, 'sgdb.db'))
    df = pd.read_sql_query("SELECT * FROM names", conn)
    df.to_parquet(names_parquet, index=False)
else:
    # Load the names from the parquet file
    df = pd.read_parquet(names_parquet)


In [3]:
processor = NameProcessor()

res = processor.process_names(df, 'name')

2025-06-15 17:36:07.155 | INFO     | src.preprocess.names:process_names:206 - Processing 6214472 names in column 'name'.


In [4]:
from html import unescape
res['org'] = res['org'].map(lambda x: unescape(x) if isinstance(x, str) else x)

In [5]:
# only get orgs and url for pre seeding

orgs = res[['org', 'url','date_created']]
# do

# .sort_values(by='date_created', ascending=True).drop_duplicates(subset=['org','url'], keep='first').reset_index(drop=True)

In [6]:

from urllib.parse import urlparse
from collections import deque
import re
def get_sgdi_entity_type(url):
    path = urlparse(url).path.strip('/')
    parts = path.split('/')

    if len(parts) >= 2 and parts[0].lower() == 'ministries':
        if len(parts) == 2:
            return 'ministry'
        elif len(parts) >= 4:
            if parts[-2].lower() == 'statutory-boards':
                return 'statutory board'
            elif parts[-2].lower() == 'departments':
                return 'department'
            elif parts[-2].lower() == 'others':
                return 'others'
            elif parts[-2].lower() == 'committees':
                return 'committee'
    return 'unknown'

def get_parent_url(url):
    parts = url.split('/')
    if len(parts) > 3:
        return '/'.join(parts[:-2])  # Return the URL up to the parent level
    return None

def get_sub_parent_org(string:str):
        
    pattern = re.compile(r'https://www.sgdi.gov.sg/ministries/([^/]+)/(statutory-boards|departments|committees|others)/([^/]+)')
    
    match = pattern.match(string)
    
    if match:
        parent_department_url = string.rsplit('/', 2)[0] 
        return parent_department_url
    else:
        return None
    
def get_org_parts(url: str):
    """
    Generically extracts organizational parts from an SGDI URL.

    It works by identifying a set of "marker" words and extracting the
    URL path segment that immediately follows each marker.
    """
    # Define the words that signal that the *next* segment is important.
    # Using a set for fast lookups.
    MARKER_WORDS = {
        "ministries",
        "statutory-boards",
        "departments",
        "committees",
        "others",
    }

    try:
        # 1. Safely parse the URL into its components.
        parsed_url = urlparse(url)
        # 2. Get the path and split it into a clean list of segments.
        # .strip('/') removes potential empty strings at the start or end.
        segments = parsed_url.path.strip("/").split("/")
    except Exception:
        # Handle cases where the input is not a valid URL
        return None

    extracted_parts = []
    # 3. Iterate through the segments to find our data.
    for i, segment in enumerate(segments):
        # Check if the current segment is one of our markers
        if segment in MARKER_WORDS:
            # Ensure there is a next segment to grab
            if i + 1 < len(segments):
                value = segments[i + 1]
                # We also check that the value itself isn't another marker
                if value not in MARKER_WORDS:
                    extracted_parts.append(value)

    return tuple(extracted_parts)
    
def create_org_summary(df):
    """
    Create start_date and end_date for each org
    """
    # create a copy
    df_copy = df.copy()
    df_copy['date_created'] = pd.to_datetime(df_copy['date_created'])
    
    # Group by person, position, and organization
    org_duration = df_copy.groupby(['url','org']).agg({
        'date_created': ['min', 'max'],  # Get min and max for each group
        'url': 'first',  # Get the first URL for each group
    }).reset_index()
    
    # Flatten column names
    org_duration.columns = ['url', 'org', 'start_date', 'end_date', 'first_url']
    
    # rename columns
    org_duration.rename(columns={
        'start_date': 'first_observed',
        'end_date': 'last_observed',
    }, inplace=True)

    return org_duration

def get_parent_url_robust(url):
    """
    A robust function to get the parent URL.
    Returns None if it's a top-level ministry.
    """
    # A ministry URL (e.g., .../ministries/MOF) doesn't have a parent in this context.
    if get_sgdi_entity_type(url) == "ministry":
        return None
    # For all others, the parent is the URL minus the last two path segments.
    if "/" in url:
        return url.rsplit("/", 2)[0]
    return None
def build_complete_org_hierarchy(raw_orgs_df):
    """
    Builds a complete hierarchy, creating missing parents, inferring names
    and dates, and standardizing all org names to a 'PARENT : CHILD' format.
    """
    print("1. Summarizing initial organization data...")
    df = create_org_summary(raw_orgs_df)
    # Add a flag to distinguish original orgs from inferred ones
    df["is_inferred"] = False

    print("2. Pass 1: Discovering missing parents and creating placeholders...")
    initial_map_df = df.drop_duplicates(subset="url", keep="last")
    url_to_org_name_map = initial_map_df.set_index("url")["org"]

    existing_urls = set(df["url"])
    urls_to_check = deque(df["url"])
    new_rows = []

    while urls_to_check:
        current_url = urls_to_check.popleft()
        parent_url = get_parent_url_robust(current_url)

        if parent_url and parent_url not in existing_urls:
            print(f"   - Found missing parent: {parent_url}")
            existing_urls.add(parent_url)
            urls_to_check.append(parent_url)

            # Infer the clean name without any prefixes
            inferred_name = "Unknown"
            child_name = url_to_org_name_map.get(current_url)
            if child_name and ":" in child_name:
                inferred_name = child_name.split(":", 1)[0].strip()
            else:
                parts = get_org_parts(parent_url)
                if parts:
                    inferred_name = parts[-1].upper() # Just the part, e.g., 'CD'

            new_row = {
                "url": parent_url,
                "org": inferred_name,
                "first_observed": pd.NaT,
                "last_observed": pd.NaT,
                "first_url": parent_url,
                "is_inferred": True, # Flag this row as machine-generated
            }
            new_rows.append(new_row)

    if new_rows:
        new_orgs_df = pd.DataFrame(new_rows)
        combined_df = pd.concat([df, new_orgs_df], ignore_index=True)
        print(f"\n   -> Added {len(new_rows)} new organization(s).")
    else:
        combined_df = df

    print("3. Pass 2: Recursively aggregating dates for new parents...")
    combined_df["sub_parent_org_url"] = combined_df["url"].apply(
        get_parent_url_robust
    )
    new_parent_urls = {row["url"] for row in new_rows}
    sorted_new_parent_urls = sorted(
        list(new_parent_urls), key=len, reverse=True
    )

    for parent_url in sorted_new_parent_urls:
        children_df = combined_df[
            combined_df["sub_parent_org_url"] == parent_url
        ]
        if not children_df.empty:
            min_date = children_df["first_observed"].min(skipna=False)
            max_date = children_df["last_observed"].max(skipna=False)
            parent_index = combined_df.index[combined_df["url"] == parent_url]
            combined_df.loc[parent_index, "first_observed"] = min_date
            combined_df.loc[parent_index, "last_observed"] = max_date

    print("4. Pass 3: Finalizing links and reconstructing all names...")
    # First, get the initial parent names
    unique_orgs_for_mapping = combined_df.drop_duplicates(
        subset="url", keep="last"
    )
    url_to_name_map = unique_orgs_for_mapping.set_index("url")["org"]
    combined_df["sub_parent_org_name"] = combined_df[
        "sub_parent_org_url"
    ].map(url_to_name_map)

    # --- NAME AND TYPE RECONSTRUCTION ---
    def get_base_name(org_name):
        """Extracts the 'child' part of a name."""
        if pd.isna(org_name):
            return "Unknown"
        return org_name.split(":", 1)[-1].strip()

    # Get the base name for every org before we overwrite them
    combined_df["base_name"] = combined_df["org"].apply(get_base_name)

    # Rebuild the final org name for consistency
    combined_df["org"] = combined_df.apply(
        lambda row: f"{row['sub_parent_org_name']} : {row['base_name']}"
        if pd.notna(row["sub_parent_org_name"])
        else row["org"], # Keep original name for ministries
        axis=1,
    )

    # Now, set the final entity type for inferred orgs
    combined_df["sgdi_entity_type"] = combined_df["url"].apply(
        get_sgdi_entity_type
    )
    combined_df.loc[
        combined_df["is_inferred"] == True, "sgdi_entity_type"
    ] = "inferred"

    # Clean up temporary columns
    combined_df = combined_df.drop(columns=["is_inferred", "base_name"])
    # --- END RECONSTRUCTION ---

    combined_df["parts"] = combined_df["url"].apply(get_org_parts)
    combined_df = combined_df.sort_values(by="url").reset_index(drop=True)

    print("5. Hierarchy construction complete.")
    return combined_df
transformed_org = build_complete_org_hierarchy(orgs)
    
# transformed_org['sub_parent_org_url'] = transformed_org.url.map(lambda x : get_sub_parent_org(x))
# transformed_org['parts'] = transformed_org.url.map(lambda x : get_org_parts(x))
# orgs['parent_org_url'] = orgs.url.map(lambda x : re.match(r'https://www.sgdi.gov.sg/ministries/([^/]+)', x).group(0) if re.match(r'https://www.sgdi.gov.sg/ministries/([^/]+)', x) else None)
# transformed_org['sgdi_entity_type'] = transformed_org['url'].map(get_sgdi_entity_type)

1. Summarizing initial organization data...
2. Pass 1: Discovering missing parents and creating placeholders...
   - Found missing parent: https://www.sgdi.gov.sg/ministries/MOF/departments/CD
   - Found missing parent: https://www.sgdi.gov.sg/ministries/MOF/departments/CD/departments/fmp
   - Found missing parent: https://www.sgdi.gov.sg/ministries/MOF/departments/agd/departments/ft/departments/building-cumulus-community
   - Found missing parent: https://www.sgdi.gov.sg/ministries/MOF/departments/agd/departments/ft
   - Found missing parent: https://www.sgdi.gov.sg/ministries/MOF/departments/agd/departments/ft/departments/projects-2
   - Found missing parent: https://www.sgdi.gov.sg/ministries/MOF/departments/agd/departments/po/departments/fcd
   - Found missing parent: https://www.sgdi.gov.sg/ministries/MOF/departments/agd/departments/po/departments/po
   - Found missing parent: https://www.sgdi.gov.sg/ministries/MOF/departments/agd/departments/ro
   - Found missing parent: https://

# Clean up names and org

In [7]:

# clean up html in org and rank
res['rank'] = res['rank'].map(lambda x: unescape(x) if isinstance(x, str) else x)
# apply autocorrect
res['rank'] = res['rank'].map(lambda x: str(x).replace('&', 'and') if isinstance(x, str) else x)

In [8]:
def create_position_tenure(df):
    """
    Create start_date and end_date for each unique position held by each person
    """
    df['date_created'] = pd.to_datetime(df['date_created'])
    
    # Group by person, position, and organization
    position_tenure = df.groupby(['lower_name', 'rank', 'org']).agg({
        'clean_name': 'first',
        'date_created': ['min', 'max'],  # Get min and max for each group
        'email': 'last',
        'tel': 'last',
        'url': 'first',
        'type': 'first',
        'raw_name': 'first'
    }).reset_index()
    
    # Flatten column names
    position_tenure.columns = [
        'lower_name', 'rank', 'org', 'clean_name',
        'start_date', 'end_date', 'email', 'tel', 'url', 'type','raw_name'
    ]
    
    # Calculate tenure duration
    position_tenure['tenure_days'] = (
        position_tenure['end_date'] - position_tenure['start_date']
    ).dt.days
    


    return position_tenure

In [9]:
tenure_df = create_position_tenure(res)

In [10]:
# save the tenure_df to a parquet file
tenure_parquet = os.path.join(data_dir, 'tenure.parquet')
# if not os.path.exists(tenure_parquet):
tenure_df.to_parquet(tenure_parquet, index=False)

# save the transformed_org to a parquet file
orgs_parquet = os.path.join(data_dir, 'orgs.parquet')
# if not os.path.exists(orgs_parquet):
transformed_org.to_parquet(orgs_parquet, index=False)
